# Description
This script's purpose is to download atmospherically corrected sentinel 2 satellite imagery from sentinel hub using a bounding box.


In [ ]:
from tqdm import tqdm
from sentinelhub import (
    SHConfig, BBox, CRS, SentinelHubRequest, BBoxSplitter,
    MimeType, DataCollection, bbox_to_dimensions, read_data
)
# from shapely.geometry import shape, Polygon, MultiPolygon, MultiLineString

In [ ]:
# Write your credentials here if you haven't already put them into config.json
CLIENT_ID = ''
CLIENT_SECRET = ''

config = SHConfig()
if CLIENT_ID and CLIENT_SECRET:
    config.sh_client_id = CLIENT_ID
    config.sh_client_secret = CLIENT_SECRET

In [ ]:
_bbox = BBox([2.087283,8.960042,3.099352,9.955682], crs=CRS.WGS84)
bbox_area = _bbox.geometry
# bbox_area = bbox_area["features"][0]["geometry"]
print(f"Type of area is {type(bbox_area)}")
bbox_splitter = BBoxSplitter([bbox_area], CRS.WGS84, (5, 5))
bboxes_ = bbox_splitter.get_bbox_list()
print(f"Number of beat-boxes is: {len(bboxes_)}")

In [ ]:
# http://bboxfinder.com/ NOTE: Pass in geojson to that site to get coordinates below

# _bbox = BBox([2.087283,8.960042,3.099352,9.955682], crs=CRS.WGS84)

time_interval = '2019-11-20T00:00:00Z', '2019-11-21T00:00:00Z'

evalscript_true_color = """
//VERSION=3

function setup() {
    return {
        input: [{
            bands: ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12"],
        }],
        output: {
            bands: 12
        }
    };
}

function evaluatePixel(sample) {
    return [sample.B01, sample.B02, sample.B03, sample.B04, sample.B05, sample.B06, sample.B07, sample.B08, sample.B8A, sample.B09, sample.B11, sample.B12];
}
"""
for i, _bbox in tqdm(enumerate(bboxes_)):
    bbox_size = bbox_to_dimensions(_bbox, resolution=10)
    request = SentinelHubRequest(
        data_folder="20191120_20191121",
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A,
                time_interval=time_interval,
            )
        ],
        responses=[
            SentinelHubRequest.output_response('default', MimeType.TIFF)
        ],
        bbox=_bbox,
        size=bbox_size,
        config=config
    )

    # image = request.get_data()[0]
    request.save_data()